In [65]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [66]:
df = pd.read_csv("Online Retail.csv")

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [67]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [68]:
df_pivot = df.pivot_table(index=['CustomerID'],
                columns=['StockCode'],
                values='Quantity',
                fill_value=0).reset_index()

In [69]:
purchase_counts = df_pivot.groupby('CustomerID').sum()

In [70]:
customer_similarities = pd.DataFrame(cosine_similarity(purchase_counts), index=purchase_counts.index, columns=purchase_counts.index)

In [71]:
def recommend_products(customer_id, n_recommendations=5):
    similar_customers = customer_similarities[customer_id].sort_values(ascending=False).index[1:]
    recommended_products = set()

    for similar_customer in similar_customers:
        products_bought = purchase_counts.loc[similar_customer][purchase_counts.loc[similar_customer] > 0].index
        products_not_bought = products_bought.difference(purchase_counts.loc[customer_id][purchase_counts.loc[customer_id] > 0].index)
        recommended_products.update(products_not_bought)

        if len(recommended_products) >= n_recommendations:
            break

    return list(recommended_products)[:n_recommendations]

In [72]:
def compareRecommendations(cID, prod_rec):
    comparison = {'Products Purchased': [], 'Products Recommended': []}
    comparison['Products Purchased'] = list(df[df['CustomerID'] == cID]['Description'])
    comparison['Products Recommended'] = set([list(df[df['StockCode'] == prodID]['Description'])[0] for prodID in prod_rec])
    return comparison

In [73]:
testID = np.random.choice(df['CustomerID'])
compareRecommendations(testID, recommend_products(testID, 5))

KeyError: nan